# Process chat data from the wotkshops related to "Data Discovery using Conversational Agents"

## Split the raw text to questions and answers

### 1. London workshop using Bard

Create a function to read text between specific characters

In [5]:
import pandas as pd
import os
import csv

In [6]:

def take_text(f_read, q_start, q_end):
    copy = False
    txt_lines=[]# store the lines between the text
    gathered_lines=[]# merge the lines to one string
    for line in f_read:
        if line == q_start:
            copy = True
            continue
        elif line == q_end:
            copy = False
            if len(txt_lines)>0:
                gathered_lines.append(" ".join(txt_lines))# merge the lines to one stirng
            txt_lines=[]# clean the list to store the next lines
            continue
        elif copy:
            # f_save.write(line)
            txt_lines.append(line)

    return gathered_lines
    

Read the txt files in a loop and then save all questions and answers in a csv

In [9]:

path='path_to_saved_files'

df_all=pd.DataFrame()

for file in os.listdir(str(path)):
    #read the txt files
    with open(str(path)+str(file), 'r', encoding='utf-8') as f_read:
        lines = f_read.readlines()# load the file
        lines1=[i for i in lines if i != '\n']# remove all null lines
        # remove the lines where Bard suggest multiple windows
        for i in range(len(lines1)):
            if lines1[i]== 'Search related topics\n':
                lines1.pop(i)
                lines1.pop(i)
                break
        lines2=lines1[:lines1.index('mic\n')] # remove the end lines
        question1=take_text(lines2,'Opens in a new window\n','edit\n')# take the first question
        question2=take_text(lines2,'more_vert\n','edit\n')# take the rest questions
        questions=question1+question2
        answers=take_text(lines2,'edit\n','thumb_upthumb_down\n')


# print([file[:-4]]*len(questions))  
# print(range(len(questions)))      
        #save to df
        df=pd.DataFrame({'name':[file[:-4]]*len(questions),'sequence':range(1,len(questions)+1),'question':[x.strip() for x in questions],'answer':[x.strip() for x in answers]})
        
        df_all=pd.concat([df_all,df])
        

df_all.to_csv('/London_Bard_text.csv',index=None, encoding='utf-8')

### 2. Berlin workshop using ChatGPT

Function to read text between specific characters

In [2]:
def take_text(f_read, q_start, q_end):
    copy = False
    txt_lines=[]# store the lines between the text
    gathered_lines=[]# merge the lines to one string
    for line in f_read:
        if line == q_start:
            copy = True
            continue
        elif line == q_end:
            copy = False
            if len(txt_lines)>0:
                gathered_lines.append(" ".join(txt_lines))# merge the lines to one stirng
            txt_lines=[]# clean the list to store the next lines
            continue
        elif copy:
            # f_save.write(line)
            txt_lines.append(line)

    return gathered_lines

Read the txt files and save questosn and answers into csv

In [4]:
path='path_to_saved_files'

df_all=pd.DataFrame()

for file in os.listdir(str(path)):
    #read the txt files
    with open(str(path)+str(file), 'r', encoding='utf-8') as f_read:
        lines = f_read.readlines()# load the file
        question1=[lines[0]]# take the first question
        question2=take_text(lines,'\n','ChatGPT\n')# take the rest questions
        questions=question1+question2
        answers=take_text(lines,'ChatGPT\n','\n')

# print([file[:-4]]*len(questions))  
# print(range(len(questions)))      
        #save to df
        df=pd.DataFrame({'name':[file[:-4]]*len(questions),'sequence':range(1,len(questions)+1),'question':[x.strip() for x in questions],'answer':[x.strip() for x in answers]})
        
        df_all=pd.concat([df_all,df])
        

df_all.to_csv('/Berlin_ChatGPT_text.csv',index=None, encoding='utf-8')

### 3. Pilot workshop ChatGPT

These data did not have a unified format and their process was manual.

## Descriptive statistics for questions

Function to tokenize the text in queries and answers

In [11]:
import pandas as pd
from spacy.lang.en import English
from spacy.lang.de import German
import string


path='path'

nlpE = English()
nlpG= German()

# Create list of word tokens
def tokenize(text, nlp):
    text_no_punc=text.translate(str.maketrans('', '', string.punctuation))# remove punctuation
    # print(text)
    # print(text_no_punc)
    ## tokenize
    my_doc = nlp(text_no_punc)
    token_list = []
    for token in my_doc:
        token_list.append(token.text)
    return token_list


Create a file with descriptive stats for the Berlin workshop. 

(number of words for queries and asnwers, language, agent)

In [3]:
#============= Berlin workshop ChatGPT =======================
df=pd.read_csv('/Berlin_ChatGPT_text.csv', encoding='utf-8')

q=[len(tokenize(x, nlpG)) for x in df['question']]# store the length of questions in a list
a=[len(tokenize(x, nlpG)) for x in df['answer']]# store the length of answers in a list
names=[str(x)+'_Berlin' for x in df['name']]# create ID
language=['German']*len(q)
agent=['ChatGPT4']*len(q)
group=['Berlin']*len(q)

df_words=pd.DataFrame({'name_ID':names,'language':language,'agent':agent,'group':group,'num_words_question':q,'num_words_answer':a})# create a dataframe with the new data
df_new=pd.concat([df,df_words], axis=1)# merge two df
df_new.to_csv(str(path)+'Berlin_ChatGPT_chat_stats.csv', index=None, encoding='utf-8')# save new data

['\ufeffSuche nach einem Datensatz zur Nutzung von Open Source tools in Europa von 1990-2022', 'Nenne mir weitere Quellen, wo ich einen aktuellen Datensatz Zur open Source Nutzung in Europa finden kann?', 'Nenne mir weitere Quellen, die sich nur mit Open source Trends befassen in der EU', 'Was ist der aktuelle Trend zur Nutzung von Open Source? Welche Quellen kennst du?', 'Nenne mir einen Report, der sich mit der Statistik von Open Source befasst mit Quellenangabe', 'Nenne mir den Link von Octoverse, worüber ich den Report direkt finden kann', 'Welchen Trend zu Open source beobachtest du in der EU? Nenne mir Quellen mit Links.', 'Du hattest mir vorhin als Quelle diesen Link vorgeschlagen https://octoverse.github.com/ . Hast du noch weitere Links zur Nutzung von Open Source?', 'Nenne mir bitte weitere URL, die einen Trend zur Nutzung von Open Source tools zeigen', 'Nenne mir eine URL zur Open Source Nutzung in Deutschland', 'Nenne mir einen konkreten Report zur Open source Nutzung in De

Create a file with descriptive stats for the pilot study

In [12]:
#============= Pilot workshop ChatGPT =======================
df=pd.read_csv(str(path)+'Pilot_ChatGPT_text.csv', encoding='utf-8')
q=[len(tokenize(x, nlpE)) for x in df['question']]# store the length of questions in a list
a=[len(tokenize(x, nlpE)) for x in df['answer']]# store the length of answers in a list
names=[str(x)+'_Pilot' for x in df['name']]# create ID
language=['English']*len(q)
agent=['ChatGPT3.5']*len(q)
group=['Pilot']*len(q)

df_words=pd.DataFrame({'name_ID':names,'language':language,'agent':agent,'group':group,'num_words_question':q,'num_words_answer':a})# create a dataframe with the new data
df_new=pd.concat([df,df_words], axis=1)# merge two df
df_new.to_csv(str(path)+'Pilot_ChatGPT_chat_stats.csv', index=None, encoding='utf-8')# save new data

Create a file with descriptive stats for the London workshop.

In [12]:
#============= London workshop Bard =======================
df=pd.read_csv(str(path)+'London_Bard_text.csv', encoding='utf-8')
q=[len(tokenize(x, nlpE)) for x in df['question']]# store the length of questions in a list
a=[len(tokenize(x, nlpE)) for x in df['answer']]# store the length of answers in a list
names=[str(x)+'London' for x in df['name']]# create ID
language=['English']*len(q)
agent=['Bard']*len(q)
group=['London']*len(q)

df_words=pd.DataFrame({'name_ID':names,'language':language,'agent':agent,'group':group,'num_words_question':q,'num_words_answer':a})# create a dataframe with the new data
df_new=pd.concat([df,df_words], axis=1)# merge two df
df_new.to_csv(str(path)+'London_Bard_chat_stats.csv', index=None, encoding='utf-8')# save new data

Following the above we manulaly added some lables, such as familiarisation with the agent, success of the session, usefullness. This came from the questionaires of the study.

## Desciptive statistics for each chat

We calculate the total number of queries, the mena number of wrods per query and answer.

In [16]:
import pandas as pd

path='path'


dfB=pd.read_csv(str(path)+'Berlin_ChatGPT_chat_stats_questionaires.csv', encoding='utf-8')
dfP=pd.read_csv(str(path)+'Pilot_ChatGPT_chat_stats_questionaires.csv', encoding='utf-8')
dfL=pd.read_csv(str(path)+'London_Bard_chat_stats_questionaires.csv', encoding='utf-8')

df=pd.concat([dfB,dfP,dfL])

names=df['name_ID'].unique()# find the unique participants

name_ID=[]
q_num=[]
q_mean=[]
a_mean=[]
language=[]
agent=[]
group=[]
familiar=[]
sunccess=[]
useful=[]

for p in names:
    data=df[df.loc[:,'name_ID']==p]
    name_ID.append(data['name_ID'].iloc[0])
    q_num.append(len(data))
    q_mean.append(data['num_words_question'].mean())
    a_mean.append(data['num_words_answer'].mean())
    language.append(data['language'].iloc[0])
    agent.append(data['agent'].iloc[0])
    group.append(data['group'].iloc[0])
    familiar.append(data['familiar_with_agent'].iloc[0])
    sunccess.append(data['success_of_session'].iloc[0])
    useful.append(data['usefull'].iloc[0])



df_new=pd.DataFrame({'name_ID':name_ID, 'num_questions':q_num, 'mean_question_length':q_mean, 'mean_answer_length':a_mean, 
                     'langugae':language, 'agent':agent, 'group':group,'familiar_with_agent':familiar,'success_of_session':sunccess,'useful':useful })


df_new.to_csv(str(path)+'chat_stats_v2.csv', index=None, encoding='utf-8')
